In [1]:
import pandas as pd

df = pd.read_csv('data/game_scores_homepage.csv')

In [2]:
df.head()

,날짜,요일,경기시간,장소,팀1명,팀2명,홈/원정,게임결과,팀1 점수,팀2 점수,...,팀2 3이닝점수,팀2 4이닝점수,팀2 5이닝점수,팀2 6이닝점수,팀2 7이닝점수,팀2 8이닝점수,팀2 9이닝점수,팀2 10이닝점수,팀2 11이닝점수,팀2 12이닝점수
0,2024-03-23,(토),14:00,잠실,한화,LG,원정,패,2,8,...,0,1,0,-,-,-,-,-,-,-
1,2024-03-24,(일),14:00,잠실,한화,LG,원정,승,8,4,...,4,0,1,2,-,-,-,-,-,-
2,2024-03-26,(화),18:30,문학,한화,SSG,원정,승,6,0,...,2,0,0,0,-,-,-,-,-,-
3,2024-03-27,(수),18:30,문학,한화,SSG,원정,승,3,1,...,0,0,0,0,-,-,-,-,-,-
4,2024-03-28,(목),18:30,문학,한화,SSG,원정,승,10,6,...,0,1,1,0,-,-,-,-,-,-


In [4]:
# 팀별 승률 계산 함수
def calculate_win_rate(team, df, is_home=None):
    """
    팀별 승률을 계산하는 함수.
    is_home: True이면 홈 경기만, False이면 원정 경기만 필터링.
    """
    # 팀이 '팀1명' 또는 '팀2명'에 포함된 경기들만 필터링
    team_games = df[(df['팀1명'] == team) | (df['팀2명'] == team)].copy()
    
    if is_home is not None:
        # 홈 경기인 경우 '홈' 또는 원정 경기인 경우 '원정' 필터링
        team_games = team_games[team_games['홈/원정'] == ('홈' if is_home else '원정')]

    # 승리 여부를 계산
    team_games['승리'] = team_games.apply(
        lambda row: (row['팀1명'] == team and row['게임결과'] == '승') or
                    (row['팀2명'] == team and row['게임결과'] == '승'), axis=1)
    
    # 승리한 경기 수와 전체 경기 수 계산
    total_games = len(team_games[team_games['게임결과'] != '무'])  # 무승부 제외
    won_games = team_games['승리'].sum()
    
    # 승률 계산
    win_rate = won_games / total_games if total_games > 0 else 0
    
    return win_rate

# 각 팀별 승률 계산 (전체 승률, 홈 승률, 원정 승률)
teams = ['KIA', '삼성','LG', '두산', 'KT', 'SSG', '롯데', 'NC', '키움']

# 빈 데이터프레임 생성
winning_rate_df = pd.DataFrame(columns=['팀명', '승률', '홈 승률', '원정 승률'])

for team in teams:
    total_win_rate = calculate_win_rate(team, df)  # 전체 승률
    home_win_rate = calculate_win_rate(team, df, is_home=True)  # 홈 승률
    away_win_rate = calculate_win_rate(team, df, is_home=False)  # 원정 승률

    # 결과를 DataFrame 형태로 만들어서 concat으로 추가
    team_result = pd.DataFrame({
        '팀명': [team],
        '승률': [total_win_rate * 100],  # 퍼센트로 변환
        '홈 승률': [home_win_rate * 100],    # 퍼센트로 변환
        '원정 승률': [away_win_rate * 100]   # 퍼센트로 변환
    })
    
    # 기존 DataFrame과 합치기
    winning_rate_df = pd.concat([winning_rate_df, team_result], ignore_index=True)

C:\Users\82102\AppData\Local\Temp\ipykernel_27192\976661958.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  winning_rate_df = pd.concat([winning_rate_df, team_result], ignore_index=True)


In [5]:
# 합계를 추가할 행 계산
total_row = {
    '팀명': '전체',
    '승률': winning_rate_df['승률'].mean(),
    '홈 승률': winning_rate_df['홈 승률'].mean(),
    '원정 승률': winning_rate_df['원정 승률'].mean()
}

# total_row를 DataFrame으로 변환
total_row_df = pd.DataFrame([total_row])

# 기존 DataFrame과 합치기 (pd.concat 사용)
winning_rate_df = pd.concat([winning_rate_df, total_row_df], ignore_index=True)

# 소수점 2자리로 반올림
winning_rate_df = winning_rate_df.round(2)

In [6]:
winning_rate_df

,팀명,승률,홈 승률,원정 승률
0,KIA,31.25,22.22,42.86
1,삼성,37.50,42.86,33.33
2,LG,50.00,44.44,57.14
3,두산,62.50,57.14,66.67
4,KT,62.50,57.14,66.67
5,SSG,62.50,28.57,88.89
6,롯데,43.75,71.43,22.22
7,NC,28.57,14.29,42.86
8,키움,37.50,55.56,14.29
9,전체,46.23,43.74,48.32


In [7]:
winning_rate_df.to_csv('data/승률계산결과.csv')